In [139]:
import ezsheets
import math
import pprint
import datetime
import dateparser


def addRevision(date, page, wordMistakes, lineMistakes, currentInterval):
    None


def getScore(wordMistakes, lineMistakes):
    return wordMistakes * 1 + lineMistakes * 4


def getNextInterval(page,score, currentInterval):
    # Convert score into an interval delta
    if score == 0:  # Perfect page
        intervalDelta = +3
    elif score == 1:  # 1 Word Mistake
        intervalDelta = +2
    elif score <= 3:  # 3 Word Mistakes
        intervalDelta = +1
    elif score == 4:  # 1 Line Mistake
        intervalDelta = 0
    elif score <= 8:  # 2 Line Mistakes
        intervalDelta = -1
    elif score <= 12:  # 3 Line Mistakes
        intervalDelta = -2
    elif score <= 20:  # 5 Line Mistakes
        intervalDelta = -3
    elif score <= 30:  # 7.5 Line Mistakes - Half a page
        intervalDelta = -5
    else:  # More than half a page
        intervalDelta = -7

    nextInterval = currentInterval + intervalDelta

    # If Score is 8 or more (2 line mistakes or more), then we have to restrict the max Interval
    if score == 4:  # 1 Line Mistake - 40 days max
        maxInterval = 40
    elif score <= 8:  # 2 Line Mistakes - 30 days/1 month max
        maxInterval = 30
    elif score <= 12:  # 3 Line Mistakes - 3 weeks max
        maxInterval = 21
    elif score <= 20:  # 5 Line Mistakes - 2 weeks max
        maxInterval = 14
    elif score <= 30:  # 7.5 Line Mistakes - Half a page - 1 week max
        maxInterval = 7
    else:  # More than half a page - 3 days max
        maxInterval = 3

    print(
        f"page={page}, score = {score}, currentInterval={currentInterval}, intervalDelta={intervalDelta}, nextInterval={nextInterval}, maxInterval={maxInterval}"
    )
    return nextInterval if maxInterval > nextInterval else maxInterval

def addRevision(revisionDate,page,wordMistakes,lineMistakes,currentInterval):
    score = getScore(wordMistakes, lineMistakes)
    nextInterval = getNextInterval(page,score, currentInterval)
    
    dueDate = revisionDate + datetime.timedelta(days=nextInterval)

    revisionList.append( {
        "revisionDate": revisionDate,
        "page": page,
        "wordMistakes": wordMistakes,
        "lineMistakes": lineMistakes,
        "score": score,
        "currentInterval": currentInterval,
        "nextInterval": nextInterval,
        "dueDate": dueDate,
    })

In [140]:
# Get the first sheet that contains the form responses for Quran Review data
reviewSheet = ezsheets.Spreadsheet('114LWy42iuyzIfKFrg-egp2o6jT-2XrnkzrYFSqmweOE')[0]
rows = reviewSheet.getRows(2)
#print(rows)
#rows = [
#    ["11/12/2019 10:31:49", "418", "1", "", ""],
#    ["11/12/2019 14:24:29", "469", "2", "0", "1"],
#]

In [141]:
rows

[['11/12/2019 10:31:49', '418', '1', '', ''],
 ['11/12/2019 14:24:29', '469', '2', '0', '1'],
 ['11/12/2019 14:29:13', '436', '0', '', '22'],
 ['11/12/2019 16:57:34', '470-471', '0', '', '3'],
 ['11/12/2019 17:00:31', '472', '1', '', '3'],
 ['11/12/2019 17:08:03', '473', '1', '2', '3'],
 ['11/12/2019 17:15:02', '474', '', '1', '3'],
 ['11/12/2019 17:20:48', '475', '1', '2', ''],
 ['11/12/2019 17:27:44', '476', '0', '', '3'],
 ['11/12/2019 17:34:45', '477', '1', '', '3'],
 ['11/13/2019 3:47:10', '478-479', '0', '', ''],
 ['11/13/2019 3:56:53', '418', '3', '', '30'],
 ['11/13/2019 4:06:18', '419', '4', '4', '30'],
 ['11/15/2019 2:17:05', '420', '1', '2', '30'],
 ['11/15/2019 2:44:08', '421', '3', '', '30'],
 ['11/15/2019 2:50:56', '422', '5', '', '30'],
 ['11/15/2019 12:55:24', '423', '2', '0', '30'],
 ['11/15/2019 12:58:29', '424', '0', '', '30'],
 ['11/15/2019 13:34:17', '425', '2', '3', '30'],
 ['11/20/2019 21:02:55', '482-483', '0', '', ''],
 ['11/22/2019 12:56:29', '453', '4', '', '

In [142]:
revisionList = []

for index, row in enumerate(rows):
    if row[0] != "":
        # Assign the columns to proper variable names
        # revisionDate = dt.datetime.strptime(row[0],'%m/%d/%Y %H:%M:%S')
        # No need to mention format string with dateutil.parser
        revisionDate = dateparser.parse(row[0]).date()
        page = row[1]

        # since empty string can be there, we can ask int function to return 0 if it finds an empty string
        wordMistakes = int(row[2] or 0)
        lineMistakes = int(row[3] or 0)
        currentInterval = int(row[4] or 0)

        if page.isdecimal():
            addRevision(revisionDate,page,wordMistakes,lineMistakes,currentInterval)
        else:
            print("page is not number:", page)
            # Get the start and end page number from the combined string which is in this format "420-425"
            startPage,endPage = page.split("-")
            #Now parse the page Numbers
            startPage = int(startPage or 0)
            endPage = int(endPage or 0)
            for i in range(startPage, endPage+1):
                addRevision(revisionDate,str(i),wordMistakes,lineMistakes,currentInterval)

page=418, score = 1, currentInterval=0, intervalDelta=2, nextInterval=2, maxInterval=30
page=469, score = 2, currentInterval=1, intervalDelta=1, nextInterval=2, maxInterval=30
page=436, score = 0, currentInterval=22, intervalDelta=3, nextInterval=25, maxInterval=30
page is not number: 470-471
page=470, score = 0, currentInterval=3, intervalDelta=3, nextInterval=6, maxInterval=30
page=471, score = 0, currentInterval=3, intervalDelta=3, nextInterval=6, maxInterval=30
page=472, score = 1, currentInterval=3, intervalDelta=2, nextInterval=5, maxInterval=30
page=473, score = 9, currentInterval=3, intervalDelta=-2, nextInterval=1, maxInterval=21
page=474, score = 4, currentInterval=3, intervalDelta=0, nextInterval=3, maxInterval=40
page=475, score = 9, currentInterval=0, intervalDelta=-2, nextInterval=-2, maxInterval=21
page=476, score = 0, currentInterval=3, intervalDelta=3, nextInterval=6, maxInterval=30
page=477, score = 1, currentInterval=3, intervalDelta=2, nextInterval=5, maxInterval=30

In [217]:
revisionList

[{'revisionDate': datetime.date(2019, 11, 12),
  'page': '418',
  'wordMistakes': 1,
  'lineMistakes': 0,
  'score': 1,
  'currentInterval': 0,
  'nextInterval': 2,
  'dueDate': datetime.date(2019, 11, 14)},
 {'revisionDate': datetime.date(2019, 11, 12),
  'page': '469',
  'wordMistakes': 2,
  'lineMistakes': 0,
  'score': 2,
  'currentInterval': 1,
  'nextInterval': 2,
  'dueDate': datetime.date(2019, 11, 14)},
 {'revisionDate': datetime.date(2019, 11, 12),
  'page': '436',
  'wordMistakes': 0,
  'lineMistakes': 0,
  'score': 0,
  'currentInterval': 22,
  'nextInterval': 25,
  'dueDate': datetime.date(2019, 12, 7)},
 {'revisionDate': datetime.date(2019, 11, 12),
  'page': '470',
  'wordMistakes': 0,
  'lineMistakes': 0,
  'score': 0,
  'currentInterval': 3,
  'nextInterval': 6,
  'dueDate': datetime.date(2019, 11, 18)},
 {'revisionDate': datetime.date(2019, 11, 12),
  'page': '471',
  'wordMistakes': 0,
  'lineMistakes': 0,
  'score': 0,
  'currentInterval': 3,
  'nextInterval': 6,
  

In [213]:
pageSummaryMap ={}
pageDueMap ={}

for rev in revisionList:
    page = rev['page']
    dueDate = rev['dueDate']
    
    if page in pageSummaryMap:
        print("page found:",page)
        pageMapCurrent=pageSummaryMap.get(page)
        pageMapCurrent["dueDate"]=dueDate
        revisionCount = pageMapCurrent["revisionCount"]
        totalScore = pageMapCurrent["averageScore"]*revisionCount + rev['score']
        
        pageMapCurrent["revisionCount"] = revisionCount + 1
        pageMapCurrent["averageScore"] = totalScore/(revisionCount + 1)
        pageMapCurrent["interval"] = rev['nextInterval']
        
    else:
        newPageMap={
        #"page":page,
        "dueDate":dueDate,
        "revisionCount":1,
        "averageScore":rev['score'],
        "interval":rev['nextInterval']
        }
        pageSummaryMap.setdefault(page,newPageMap)
    
    pageDueMap[page]=dueDate

len(pageSummaryMap)

page found: 418


25

In [218]:
pageSummaryMap

{'418': {'dueDate': datetime.date(2019, 12, 13),
  'revisionCount': 2,
  'averageScore': 2.0,
  'interval': 30},
 '469': {'dueDate': datetime.date(2019, 11, 14),
  'revisionCount': 1,
  'averageScore': 2,
  'interval': 2},
 '436': {'dueDate': datetime.date(2019, 12, 7),
  'revisionCount': 1,
  'averageScore': 0,
  'interval': 25},
 '470': {'dueDate': datetime.date(2019, 11, 18),
  'revisionCount': 1,
  'averageScore': 0,
  'interval': 6},
 '471': {'dueDate': datetime.date(2019, 11, 18),
  'revisionCount': 1,
  'averageScore': 0,
  'interval': 6},
 '472': {'dueDate': datetime.date(2019, 11, 17),
  'revisionCount': 1,
  'averageScore': 1,
  'interval': 5},
 '473': {'dueDate': datetime.date(2019, 11, 13),
  'revisionCount': 1,
  'averageScore': 9,
  'interval': 1},
 '474': {'dueDate': datetime.date(2019, 11, 15),
  'revisionCount': 1,
  'averageScore': 4,
  'interval': 3},
 '475': {'dueDate': datetime.date(2019, 11, 10),
  'revisionCount': 1,
  'averageScore': 9,
  'interval': -2},
 '476'

In [145]:
dueDateList = sorted(pageDueMap.values())

In [146]:
dueDateMap ={}
for dueDate in dueDateList:
    dueDateMap[dueDate] = []

In [147]:
dueDateMap

{datetime.date(2019, 11, 10): [],
 datetime.date(2019, 11, 13): [],
 datetime.date(2019, 11, 14): [],
 datetime.date(2019, 11, 15): [],
 datetime.date(2019, 11, 16): [],
 datetime.date(2019, 11, 17): [],
 datetime.date(2019, 11, 18): [],
 datetime.date(2019, 11, 21): [],
 datetime.date(2019, 11, 22): [],
 datetime.date(2019, 11, 23): [],
 datetime.date(2019, 11, 27): [],
 datetime.date(2019, 11, 29): [],
 datetime.date(2019, 12, 6): [],
 datetime.date(2019, 12, 7): [],
 datetime.date(2019, 12, 13): [],
 datetime.date(2019, 12, 14): [],
 datetime.date(2019, 12, 15): []}

In [148]:
for page in pageSummaryMap.keys():
    dueDate = pageSummaryMap[page]['dueDate']
    dueDateMap[dueDate].append(page)
dueDateMap

{datetime.date(2019, 11, 10): ['475'],
 datetime.date(2019, 11, 13): ['473'],
 datetime.date(2019, 11, 14): ['469'],
 datetime.date(2019, 11, 15): ['474'],
 datetime.date(2019, 11, 16): ['478', '479'],
 datetime.date(2019, 11, 17): ['472', '477'],
 datetime.date(2019, 11, 18): ['470', '471', '476'],
 datetime.date(2019, 11, 21): ['454'],
 datetime.date(2019, 11, 22): ['453', '455'],
 datetime.date(2019, 11, 23): ['482', '483'],
 datetime.date(2019, 11, 27): ['419'],
 datetime.date(2019, 11, 29): ['425'],
 datetime.date(2019, 12, 6): ['420'],
 datetime.date(2019, 12, 7): ['436'],
 datetime.date(2019, 12, 13): ['418'],
 datetime.date(2019, 12, 14): ['422'],
 datetime.date(2019, 12, 15): ['421', '423', '424']}

In [149]:
today = datetime.date.today()

In [150]:
pagesDueList = []
for dueDate in dueDateMap.keys():
    if dueDate <= today:
        for duePage in dueDateMap[dueDate]:
            pagesDueList.append(duePage)
        

In [202]:
outputSS = ezsheets.Spreadsheet('1rHQ5WmyFz79sG8W2pNZmr6AJDuLuQEeOjSahPyPcb5o')

In [203]:
columnNames = list(next(iter(pageSummaryMap.values())).keys())

In [204]:
columnNames.insert(0,'page')

In [205]:
print(columnNames)

['page', 'dueDate', 'revisionCount', 'averageScore', 'interval']


In [207]:
outputSheet = outputSS[0]
outputSheet.updateRow(2, ['4175', '2019-11-10', '1', '9', '-2'])

In [215]:
outputSheet = outputSS[0]

In [216]:
outputSheet

<Sheet sheetId=0, title='Due', rowCount=999, columnCount=5>

In [209]:
#First write the due pages in the first sheet
outputSheet = outputSS[0]

outputSheet.updateRow(1, columnNames)
for index, page in enumerate(sorted(pagesDueList)):
    outputRow = [page]
    for value in pageSummaryMap.get(page).values():
        outputRow.append(str(value))
    print(outputRow)
    #Google sheet's row number starts at 1. First row is the column name. Hence we have start at row 2
    outputSheet.updateRow(index + 2, outputRow)

['475', '2019-11-10', '1', '9', '-2']
['473', '2019-11-13', '1', '9', '1']
['469', '2019-11-14', '1', '2', '2']
['474', '2019-11-15', '1', '4', '3']
['478', '2019-11-16', '1', '0', '3']
['479', '2019-11-16', '1', '0', '3']
['472', '2019-11-17', '1', '1', '5']
['477', '2019-11-17', '1', '1', '5']
['470', '2019-11-18', '1', '0', '6']
['471', '2019-11-18', '1', '0', '6']
['476', '2019-11-18', '1', '0', '6']
['454', '2019-11-21', '1', '6', '-1']
['453', '2019-11-22', '1', '4', '0']
['455', '2019-11-22', '1', '4', '0']
['482', '2019-11-23', '1', '0', '3']
['483', '2019-11-23', '1', '0', '3']
['418', '2019-12-13', '2', '2.0', '30']
['419', '2019-11-27', '1', '20', '14']
['420', '2019-12-06', '1', '9', '21']
['421', '2019-12-15', '1', '3', '30']
['422', '2019-12-14', '1', '5', '29']
['423', '2019-12-15', '1', '2', '30']
['424', '2019-12-15', '1', '0', '30']
['425', '2019-11-29', '1', '14', '14']
['436', '2019-12-07', '1', '0', '25']
['453', '2019-11-22', '1', '4', '0']
['454', '2019-11-21', '

In [212]:
#Then write the all pages in the second sheet
outputSheet = outputSS[1]
outputSheet.updateRow(1, columnNames)

pagesList = sorted(list(pageSummaryMap.keys()))

for index, page in enumerate(pagesList):
    outputRow = [page]
    for value in pageSummaryMap[page].values():
        outputRow.append(str(value))
    print(outputRow)
    outputSheet.updateRow(index + 2, outputRow)

['418', '2019-12-13', '2', '2.0', '30']
['419', '2019-11-27', '1', '20', '14']
['420', '2019-12-06', '1', '9', '21']
['421', '2019-12-15', '1', '3', '30']
['422', '2019-12-14', '1', '5', '29']
['423', '2019-12-15', '1', '2', '30']
['424', '2019-12-15', '1', '0', '30']
['425', '2019-11-29', '1', '14', '14']
['436', '2019-12-07', '1', '0', '25']
['453', '2019-11-22', '1', '4', '0']
['454', '2019-11-21', '1', '6', '-1']
['455', '2019-11-22', '1', '4', '0']
['469', '2019-11-14', '1', '2', '2']
['470', '2019-11-18', '1', '0', '6']
['471', '2019-11-18', '1', '0', '6']
['472', '2019-11-17', '1', '1', '5']
['473', '2019-11-13', '1', '9', '1']
['474', '2019-11-15', '1', '4', '3']
['475', '2019-11-10', '1', '9', '-2']
['476', '2019-11-18', '1', '0', '6']
['477', '2019-11-17', '1', '1', '5']
['478', '2019-11-16', '1', '0', '3']
['479', '2019-11-16', '1', '0', '3']
['482', '2019-11-23', '1', '0', '3']
['483', '2019-11-23', '1', '0', '3']
